In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

### Data preprocessing

In [ ]:
# Load dữ liệu
dataset = pd.read_csv(r'annonimized.csv')
df_th = pd.read_csv(r'th-public.csv')
df_qt = pd.read_csv(r'qt-public.csv')
df_ck = pd.read_csv(r'ck-public.csv')

In [ ]:
len(dataset)

295198

In [ ]:
# Làm sạch cột TH
df_th['TH'] = df_th['TH'].astype(str).str.replace('\xa0', '', regex=False).str.strip()
df_th = df_th[df_th['TH'] != '']
df_th['TH'] = df_th['TH'].astype(float)

# Làm sạch cột QT
df_qt['diemqt'] = df_qt['diemqt'].astype(str).str.replace('\xa0', '', regex=False).str.strip()
df_qt = df_qt[df_qt['diemqt'] != '']
df_qt['QT'] = df_qt['diemqt'].astype(float)
df_qt = df_qt.drop(columns=['diemqt'])

# Làm sạch cột CK
df_ck['CK'] = df_ck['CK'].astype(str).str.replace('\xa0', '', regex=False).str.strip()
df_ck = df_ck[df_ck['CK'] != '']
df_ck['CK'] = df_ck['CK'].astype(float)

/tmp/ipython-input-95-3462921787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_th['TH'] = df_th['TH'].astype(float)
/tmp/ipython-input-95-3462921787.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qt['QT'] = df_qt['diemqt'].astype(float)


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295198 entries, 0 to 295197
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   concat('it001',`assignment_id`)  295198 non-null  object
 1   concat('it001',`problem_id`)     295198 non-null  object
 2   concat('it001', username)        295198 non-null  object
 3   is_final                         295198 non-null  int64 
 4   status                           295198 non-null  object
 5   pre_score                        295198 non-null  int64 
 6   coefficient                      295198 non-null  int64 
 7   concat('it001',`language_id`)    295198 non-null  object
 8   created_at                       295198 non-null  object
 9   updated_at                       295198 non-null  object
 10  judgement                        295198 non-null  object
dtypes: int64(3), object(8)
memory usage: 24.8+ MB


In [ ]:
df_th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 759 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hash    759 non-null    object 
 1   TH      753 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.8+ KB


In [ ]:
df_qt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 760 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hash    760 non-null    object 
 1   QT      754 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.8+ KB


In [ ]:
df_ck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hash    761 non-null    object 
 1   CK      755 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.0+ KB


In [ ]:
#  Tiền xử lý cột judgement (JSON string)
def parse_judgement(judgement_str):
    try:
        data = json.loads(judgement_str)
        total_tests = len(data.get('times', []))
        total_wrong = data.get('verdicts', {}).get('WRONG', 0)
        total_time = sum(data.get('times', []))
        total_mem = sum(data.get('mems', []))
        return pd.Series([total_tests, total_wrong, total_time, total_mem])
    except:
        return pd.Series([0, 0, 0, 0])

dataset[['total_tests', 'total_wrong', 'total_time', 'total_mem']] = dataset['judgement'].apply(parse_judgement)

In [ ]:
dataset.rename(columns={"concat('it001', username)": "hash", "concat('it001',`assignment_id`)": "assignment_id", "concat('it001',`problem_id`)": "problem_id"}, inplace=True)

In [ ]:
dataset

,assignment_id,problem_id,hash,is_final,status,pre_score,coefficient,"concat('it001',`language_id`)",created_at,updated_at,judgement,total_tests,total_wrong,total_time,total_mem
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0.0,SCORE,0.0,100.0,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",10.0,10.0,0.0,0.0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0.0,SCORE,0.0,100.0,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",10.0,10.0,0.0,0.0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1.0,SCORE,10000.0,100.0,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1.0,SCORE,10000.0,100.0,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1.0,SCORE,10000.0,100.0,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22904,7a3cf65c4297274cdf012a95bf85c6c685619710,aa7a4e55653d7fac90885fc6db612241ec3ed769,a5b034cb2c29d312a2e5475da6174e196cfc3c1b,1.0,SCORE,10000.0,100.0,it0012,11-16 07:04:22,11-16 07:04:30,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0
22905,7a3cf65c4297274cdf012a95bf85c6c685619710,0f045ccdc2a62462f782cd597ecb932fdd0d5a97,a5b034cb2c29d312a2e5475da6174e196cfc3c1b,0.0,SCORE,0.0,100.0,it0012,11-16 07:19:09,11-16 07:20:55,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",10.0,10.0,0.0,0.0
22906,7a3cf65c4297274cdf012a95bf85c6c685619710,0f045ccdc2a62462f782cd597ecb932fdd0d5a97,a5b034cb2c29d312a2e5475da6174e196cfc3c1b,1.0,SCORE,10000.0,100.0,it0012,11-16 07:20:47,11-16 07:20:55,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0
22907,7a3cf65c4297274cdf012a95bf85c6c685619710,1ca6281c4502b3765461ce4e2f6b8b4fce4b89d8,a5b034cb2c29d312a2e5475da6174e196cfc3c1b,1.0,SCORE,10000.0,100.0,it0012,11-16 07:45:40,11-16 07:45:54,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0.0,0.0,0.0,0.0


### Huấn luyện và dự đoán để tìm điểm thực hành

In [ ]:
dataset['created_at'] = pd.to_datetime('2024-' + dataset['created_at'], errors='coerce')
dataset['updated_at'] = pd.to_datetime('2024-' + dataset['updated_at'], errors='coerce')
dataset['submit_delay'] = (dataset['updated_at'] - dataset['created_at']).dt.total_seconds()

dataset['submit_hour'] = dataset['updated_at'].dt.hour
dataset['submit_weekday'] = dataset['updated_at'].dt.weekday

In [ ]:
dataset

,assignment_id,problem_id,hash,is_final,status,pre_score,coefficient,"concat('it001',`language_id`)",created_at,updated_at,judgement,total_tests,total_wrong,total_time,total_mem,submit_delay,submit_hour,submit_weekday
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,2024-10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",10.0,10.0,0.0,0.0,294.0,8,2
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,2024-10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",10.0,10.0,0.0,0.0,10.0,8,2
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,2024-10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0,9.0,8,2
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,2024-10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0,9.0,8,2
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,2024-10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",0.0,0.0,0.0,0.0,10.0,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,2024-01-15 16:03:53,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",0.0,0.0,0.0,0.0,10.0,16,0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,2024-01-15 16:05:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",0.0,0.0,0.0,0.0,61.0,16,0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,2024-01-15 16:05:08,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",0.0,0.0,0.0,0.0,10.0,16,0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,2024-01-15 16:05:22,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",0.0,0.0,0.0,0.0,9.0,16,0


In [ ]:
dataset['total_mem'].value_counts()

,count
total_mem,
0.0,264269
1984.0,5440
19840.0,2850
23808.0,1195
49600.0,995
...,...
13828.0,1
11316.0,1
69192.0,1


In [ ]:
# Trích xuất đặc trưng cho từng sinh viên để dự đoán điểm thực hành
agg_df_th = dataset.groupby('hash').agg({
    'assignment_id': lambda x: x.nunique(),
    # 'problem_id': lambda x: x.nunique(),
    'hash': 'count',
    # 'pre_score': ['mean', 'max', 'std'],
    'pre_score': 'mean',
    'coefficient': ['sum', 'mean'],
    'is_final': 'sum',
    # 'total_tests': 'sum',
    # 'total_wrong': 'sum',
    'total_mem': 'mean',
    # 'submit_delay': ['mean', 'std'],
    'submit_hour': 'mean',
    # 'submit_weekday': ['mean', 'std'],
    'status': lambda x: (x == 'SCORE').sum(),
}).reset_index()


In [ ]:
# Đặt lại tên cột
agg_df_th.columns = ['hash',
                     'num_assignments',
                    #  'num_problems',
                     'num_submissions',
                     'pre_score_mean',
                    #  'pre_score_max',
                    #  'pre_score_std',
                     'coefficient_sum',
                     'coefficient_mean',
                     'is_final_sum',
                    #  'tests_sum',
                    #  'wrong_sum',
                     'mem_mean',
                    #  'submit_delay_mean',
                    #  'submit_delay_std',
                     'submit_hour_mean',
                    #  'submit_weekday_mean',
                    #  'submit_weekday_std',
                     'score_count'
                     ]


In [ ]:
len(agg_df_th.columns)

10

In [ ]:
# Gộp với điểm thật TH
data = pd.merge(agg_df_th, df_th, on='hash')
data = data.dropna()

In [ ]:
# Tách dữ liệu
X_th = data.drop(columns=['hash', 'TH'])
y_th = data['TH']

In [ ]:
# Huấn luyện
X_train_th, X_val_th, y_train_th, y_val_th = train_test_split(X_th, y_th, train_size=0.8, random_state=42)

rf_th = RandomForestRegressor(random_state=42, n_estimators=400,
                              max_depth=10,
                              min_samples_split=20,
                              max_features=10,
                              max_leaf_nodes=50
                            )
rf_th.fit(X_train_th, y_train_th)

# Đánh giá
y_pred_rf_th = rf_th.predict(X_val_th)

r2_rf_th = r2_score(y_val_th, y_pred_rf_th)
print(f"Random Forest R² score: {r2_rf_th:.4f}")

Random Forest R² score: 0.4018


In [ ]:
feature_importances_th = pd.DataFrame({'feature': X_th.columns, 'importance': rf_th.feature_importances_})
feature_importances_th = feature_importances_th.sort_values('importance', ascending=False)
feature_importances_th

ValueError: All arrays must be of the same length

In [ ]:
agg_df_th['TH_pred'] = rf_th.predict(agg_df_th.drop(columns=['hash']))
agg_df_th['TH_pred'] = agg_df_th['TH_pred'].round(2)  # Làm tròn đến 2 chữ số thập phân

agg_df_th[['hash', 'TH_pred']].to_csv('submission_th_rfr.csv', index=False)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- mem_mean


In [ ]:
xg_th = XGBRegressor(objective='reg:squarederror',  # for regression
                    n_estimators=300,              # number of boosting rounds
                    learning_rate=0.05,            # smaller = more accurate, slower
                    max_depth=6,                   # tree depth, controls overfitting
                    subsample=0.8,                 # sample ratio of rows per tree
                    colsample_bytree=0.8,          # sample ratio of columns per tree
                    min_child_weight=5,            # larger = conservative (avoid overfitting)
                    gamma=0.1,                     # min loss reduction to split
                    reg_alpha=0.5,                # L1 regularization (sparse features)
                    reg_lambda=1.0,               # L2 regularization (default 1.0)
                    random_state=42,
                    verbosity=1
)

xg_th.fit(X_train_th, y_train_th,
          eval_set=[(X_val_th, y_val_th)],
          verbose=False
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_xg_th = xg_th.predict(X_val_th)
r2_xg_th = r2_score(y_val_th, y_pred_xg_th)
print(f"XGBoost R² score: {r2_xg_th:.4f}")

XGBoost R² score: 0.4269


In [ ]:
feature_importances = pd.DataFrame({'feature': X_th.columns, 'importance': rf_th.feature_importances_})
feature_importances = feature_importances.sort_values('importance', ascending=False)
feature_importances

In [ ]:
# Dự đoán trên toàn bộ tập agg_df_th (không chứa cột hash và TH_pred)
agg_df_th['TH_pred_xg'] = xg_th.predict(agg_df_th.drop(columns=['hash', 'TH_pred']))
agg_df_th['TH_pred_xg'] = agg_df_th['TH_pred_xg'].round(2)

agg_df_th[['hash', 'TH_pred_xg']].to_csv('submission_th_xgboost.csv', index=False)

### Huấn luyện và dự đoán để tìm điểm quá trình

In [ ]:
# Trích xuất đặc trưng cho từng sinh viên để dự đoán điểm quá trình
agg_df_qt = dataset.groupby('hash').agg({
    'assignment_id': lambda x: x.nunique(),
    # 'problem_id': lambda x: x.nunique(),
    'problem_id': 'count',
    'pre_score': ['mean', 'max', 'std'],
    'coefficient': ['sum', 'mean'],
    'is_final': 'sum',
    'total_tests': 'sum',
    'total_wrong': 'sum',
    # 'total_mem': 'mean',
    'submit_delay': ['std'],
    'submit_hour': 'mean',
    'submit_weekday': ['mean'],
    # 'status': lambda x: (x == 'SCORE').sum(),
}).reset_index()


In [ ]:
# Đặt lại tên cột
agg_df_qt.columns = ['hash',
                     'num_assignments',
                    #  'num_problems',
                     'num_submissions',
                     'pre_score_mean',
                     'pre_score_max',
                     'pre_score_std',
                     'coefficient_sum',
                     'coefficient_mean',
                     'is_final_sum',
                     'tests_sum',
                     'wrong_sum',
                    #  'mem_mean',
                    #  'submit_delay_mean',
                     'submit_delay_std',
                     'submit_hour_mean',
                     'submit_weekday_mean',
                    #  'submit_weekday_std',
                    #  'score_count'
                     ]


In [ ]:
len(agg_df_qt.columns)

15

In [ ]:
#  Gộp QT vào bảng đặc trưng
data_qt = pd.merge(agg_df_qt, df_qt, on='hash')
data_qt = data_qt.dropna()

#  Tách đặc trưng & nhãn
X_qt = data_qt.drop(columns=['hash', 'QT'])
y_qt = data_qt['QT']

In [ ]:
# Huấn luyện mô hình
X_train_qt, X_val_qt, y_train_qt, y_val_qt = train_test_split(X_qt, y_qt, random_state=42)
model_qt = RandomForestRegressor(random_state=42, n_estimators=400,
                                 max_depth=30,
                                 min_samples_split=5,
                                 max_features=10,
                                 min_impurity_decrease=0.01,
                              )
model_qt.fit(X_train_qt, y_train_qt)

#  Đánh giá
y_pred_qt = model_qt.predict(X_val_qt)
r2_qt = r2_score(y_val_qt, y_pred_qt)
print(f"R² score: {r2_qt:.4f}")

R² score: 0.2693


In [ ]:
feature_importances_qt = pd.DataFrame({'feature': X_qt.columns, 'importance': model_qt.feature_importances_})
feature_importances_qt = feature_importances_qt.sort_values('importance', ascending=False)
feature_importances_qt

,feature,importance
2,pre_score_mean,0.121735
0,num_assignments,0.116249
11,submit_hour_mean,0.111226
7,is_final_sum,0.107977
9,wrong_sum,0.103026
12,submit_weekday_mean,0.098052
10,submit_delay_std,0.081179
8,tests_sum,0.078771
4,pre_score_std,0.062100
1,num_submissions,0.042375


In [ ]:
agg_df_qt['QT_pred'] = model_qt.predict((agg_df_qt).drop(columns=['hash']))
agg_df_qt['QT_pred'] = agg_df_qt['QT_pred'].round(2)  # Làm tròn đến 2 chữ số thập phân

agg_df_qt[['hash', 'QT_pred']].to_csv('submission_qt_rfr.csv', index=False)

In [ ]:
xg_qt = XGBRegressor(objective='reg:squarederror',  # for regression
                    n_estimators=300,              # number of boosting rounds
                    learning_rate=0.05,            # smaller = more accurate, slower
                    max_depth=6,                   # tree depth, controls overfitting
                    subsample=0.8,                 # sample ratio of rows per tree
                    colsample_bytree=0.8,          # sample ratio of columns per tree
                    min_child_weight=5,            # larger = conservative (avoid overfitting)
                    gamma=0.1,                     # min loss reduction to split
                    reg_alpha=0.5,                # L1 regularization (sparse features)
                    reg_lambda=1.0,               # L2 regularization (default 1.0)
                    random_state=42,
                    verbosity=1
)

xg_qt.fit(X_train_qt, y_train_qt,
          eval_set=[(X_val_qt, y_val_qt)],
          verbose=False
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_xg_qt = xg_qt.predict(X_val_qt)
r2_xg_qt = r2_score(y_val_qt, y_pred_xg_qt)
print(f"XGBoost R² score: {r2_xg_qt:.4f}")

XGBoost R² score: 0.2351


### Huấn luyện và dự đoán để tìm điểm cuối kỳ

In [ ]:
# Trích xuất đặc trưng cho từng sinh viên để dự đoán điểm thực hành
agg_df_ck = dataset.groupby('hash').agg({
    'assignment_id': lambda x: x.nunique(),
    'problem_id': lambda x: x.nunique(),
    # 'hash': 'count',
    # 'pre_score': ['mean', 'max', 'std'],
    'pre_score': ['max', 'std'],
    'coefficient': ['sum', 'mean'],
    'is_final': 'sum',
    'total_tests': 'sum',
    'total_wrong': 'sum',
    'total_mem': 'mean',
    'submit_delay': ['mean', 'std'],
    'submit_hour': 'mean',
    'submit_weekday': ['mean', 'std'],
    'status': lambda x: (x == 'SCORE').sum(),
}).reset_index()


In [ ]:
# Đặt lại tên cột
agg_df_ck.columns = ['hash',
                     'num_assignments',
                     'num_problems',
                    #  'num_submissions',
                    #  'pre_score_mean',
                     'pre_score_max',
                     'pre_score_std',
                     'coefficient_sum',
                     'coefficient_mean',
                     'is_final_sum',
                     'tests_sum',
                     'wrong_sum',
                     'mem_mean',
                     'submit_delay_mean',
                     'submit_delay_std',
                     'submit_hour_mean',
                     'submit_weekday_mean',
                     'submit_weekday_std',
                     'score_count'
                     ]

In [ ]:
len(agg_df_ck.columns)

17

In [ ]:
#  Gộp CK vào bảng đặc trưng
data_ck = pd.merge(agg_df_ck, df_ck, on='hash')
data_ck = data_ck.dropna()

#  Tách đặc trưng & nhãn
X_ck = data_ck.drop(columns=['hash', 'CK'])
y_ck = data_ck['CK']

#  Huấn luyện mô hình
X_train_ck, X_val_ck, y_train_ck, y_val_ck = train_test_split(X_ck, y_ck, random_state=42)
model_ck = RandomForestRegressor(random_state=42, n_estimators=400,
                                 max_depth=30,
                                 min_samples_split=5,
                                 max_features=7,
                                 min_impurity_decrease=0.01
                                 )
model_ck.fit(X_train_ck, y_train_ck)

#  Đánh giá
y_pred_ck = model_ck.predict(X_val_ck)
r2_ck = r2_score(y_val_ck, y_pred_ck)
print(f"R² score CK: {r2_ck:.4f}")

R² score CK: 0.3705


In [ ]:
feature_importances_ck = pd.DataFrame({'feature': X_ck.columns, 'importance': model_ck.feature_importances_})
feature_importances_ck = feature_importances_ck.sort_values('importance', ascending=False)
feature_importances_ck

,feature,importance
9,mem_mean,0.234053
12,submit_hour_mean,0.112142
3,pre_score_std,0.092116
8,wrong_sum,0.085147
7,tests_sum,0.081803
13,submit_weekday_mean,0.058177
4,coefficient_sum,0.054319
14,submit_weekday_std,0.046226
15,score_count,0.045564
0,num_assignments,0.041425


In [ ]:
xg_ck = XGBRegressor(objective='reg:squarederror',
                    n_estimators=800,
                    learning_rate=0.005,
                    max_depth=5,
)

xg_ck.fit(X_train_ck, y_train_ck,
          eval_set=[(X_val_ck, y_val_ck)],
          verbose=False
)

xg_pred_ck = xg_ck.predict(X_val_ck)
r2_xg_ck = r2_score(y_val_ck, xg_pred_ck)
print(f"XGBoost R² score: {r2_xg_ck:.4f}")

XGBoost R² score: 0.3168


In [ ]:
#  đoán và lưu file
agg_df_ck['CK_pred'] = model_ck.predict(agg_df_ck.drop(columns=['hash']))
agg_df_ck['CK_pred'] = agg_df_ck['CK_pred'].round(2)

agg_df_ck[['hash', 'CK_pred']].to_csv('submission_ck_rfr.csv', index=False)

### Huấn luyện và dự đoán để tìm điểm TBTL

In [ ]:
df_tbtl = pd.merge(df_th, df_qt, on='hash')
df_tbtl = pd.merge(df_tbtl, df_ck, on='hash')
df_tbtl['TBTL'] = df_tbtl['TH'] * 0.3 + df_tbtl['QT'] * 0.3 + df_tbtl['CK'] * 0.4
df_tbtl['TBTL'] = df_tbtl['TBTL'].round(2)

In [ ]:
df_tbtl

,hash,TH,QT,CK,TBTL
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.0,7.5,5.5,5.95
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.5,7.0,6.5,7.25
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.0,9.0,4.5,6.60
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0,10.0,5.0,8.00
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,6.0,9.0,4.0,6.10
...,...,...,...,...,...
754,840d07858c03f80f4695056e2cc7d0c474b83a25,6.0,9.0,5.5,6.70
755,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.5,8.0,6.0,7.35
756,845acd04a77b3d1b623f255d9f9f8eae90892dab,5.5,6.5,6.0,6.00
757,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0,10.0,7.0,8.80


In [ ]:
df_tbtl.drop(columns=['TH', 'QT', 'CK'], inplace=True)

In [ ]:
df_tbtl

,hash,TBTL
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.95
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.25
2,01122b3ef7e59b84189e65985305f575d6bdf83c,6.60
3,013de369c439ab0ead8aa7da64423aa395a8be39,8.00
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,6.10
...,...,...
754,840d07858c03f80f4695056e2cc7d0c474b83a25,6.70
755,844f5db2e7e31ae51eba025480679ed7e4708ac6,7.35
756,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.00
757,8460eaaf887a6289fb156f7562fb739ba8e9629e,8.80


In [ ]:
# Trích xuất đặc trưng cho từng sinh viên để dự đoán điểm thực hành
agg_df_tb = dataset.groupby('hash').agg({
    'assignment_id': lambda x: x.nunique(),
    'problem_id': 'nunique',
    # 'hash': 'count',
    'pre_score': ['mean', 'max', 'std'],
    # 'pre_score': ['max', 'std'],
    'coefficient': ['sum', 'mean'],
    # 'is_final': 'sum',
    'total_tests': 'sum',
    'total_wrong': 'sum',
    'total_mem': 'mean',
    # 'submit_delay': ['mean', 'std'],
    'submit_hour': 'mean',
    'submit_weekday': ['mean', 'std'],
    'status': lambda x: (x == 'SCORE').sum(),
}).reset_index()


In [ ]:
# Đặt lại tên cột
agg_df_tb.columns = ['hash',
                     'num_assignments',
                     'num_problems',
                    #  'num_submissions',
                     'pre_score_mean',
                     'pre_score_max',
                     'pre_score_std',
                     'coefficient_sum',
                     'coefficient_mean',
                    #  'is_final_sum',
                     'tests_sum',
                     'wrong_sum',
                     'mem_mean',
                    #  'submit_delay_mean',
                    #  'submit_delay_std',
                     'submit_hour_mean',
                     'submit_weekday_mean',
                     'submit_weekday_std',
                     'score_count'
                     ]

In [ ]:
len(agg_df_tb.columns)

15

In [ ]:
#  Gộp CK vào bảng đặc trưng
data_tb = pd.merge(agg_df_tb, df_tbtl, on='hash')
data_tb = data_tb.dropna()

#  Tách đặc trưng & nhãn
X_tb = data_tb.drop(columns=['hash', 'TBTL'])
y_tb = data_tb['TBTL']

#  Huấn luyện mô hình
X_train_tb, X_val_tb, y_train_tb, y_val_tb = train_test_split(X_tb, y_tb, random_state=42)
model_tb = RandomForestRegressor(random_state=42, n_estimators=400,
                                 max_depth=20,
                                 min_samples_split=5,
                                 max_features=4,
                                 min_impurity_decrease=0.0001
                                 )
model_tb.fit(X_train_tb, y_train_tb)

#  Đánh giá
y_pred_tb = model_tb.predict(X_val_tb)
r2_tb= r2_score(y_val_tb, y_pred_tb)
print(f"R² score TBTL: {r2_tb:.4f}")

R² score TBTL: 0.4515


In [ ]:
feature_importances_tb = pd.DataFrame({'feature': X_tb.columns, 'importance': model_tb.feature_importances_})
feature_importances_tb = feature_importances_tb.sort_values('importance', ascending=False)
feature_importances_tb

,feature,importance
9,mem_mean,0.173017
2,pre_score_mean,0.114210
7,tests_sum,0.111405
10,submit_hour_mean,0.105155
8,wrong_sum,0.089474
13,score_count,0.071268
1,num_problems,0.063410
5,coefficient_sum,0.061052
4,pre_score_std,0.059743
11,submit_weekday_mean,0.052218


In [ ]:
# Dự đoán và lưu file
agg_df_tb['TBTL_pred'] = model_tb.predict(agg_df_tb.drop(columns=['hash']))
agg_df_tb['TBTL_pred'] = agg_df_tb['TBTL_pred'].round(2)  # Làm tròn đến 2 chữ số thập phân

agg_df_tb[['hash', 'TBTL_pred']].to_csv('submission_tbtl_rfr.csv', index=False)